In [105]:
import pandas as pd
import numpy as np
import os
import glob
from hrvanalysis import get_time_domain_features
from hrvanalysis import get_frequency_domain_features
import neurokit2 as nk
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings(action='once')

In [106]:
#Global Constants
minTimeDiff = 25 #25ms

In [107]:
#Final Analysis
#==============

def processLevel(start_signal, end_signal, sensorDataDF, level, debug=False):
    if(debug):
        print()
        #print("Processing Level: ", level)
    if end_signal != None:
        level_signal_start_time = sensorDataDF[sensorDataDF['MarkerName'] == start_signal]['Timestamp'].values[0]
        next_level_signal_start_time =  sensorDataDF[sensorDataDF['MarkerName'] == end_signal]['Timestamp'].values[0]
        level_df = sensorDataDF[sensorDataDF['Timestamp'].between(level_signal_start_time, next_level_signal_start_time)]
    else:
        #level_signal_start_time = sensorDataDF[sensorDataDF['MarkerName'] == start_signal]['Timestamp'].values[0]
        level_df = sensorDataDF.iloc[sensorDataDF.where(sensorDataDF['MarkerName'] == start_signal).first_valid_index():]
       

    level_df.reset_index(inplace = True) #As the indices from the main df is carried over
    
    first_right_press_ind = level_df.where(level_df['MarkerName'] == 'RightSwitchPressed').first_valid_index()
    level_df.iloc[first_right_press_ind:].replace(to_replace ="LeftSwitchPressed", value ="", inplace=True) #Remove left presses after right presses

    level_start_ind = level_df.where(level_df['MarkerName'] == 'LeftSwitchPressed').last_valid_index()
    level_end_ind = level_df.where(level_df.iloc[level_start_ind:]['MarkerName'] == 'RightSwitchPressed').first_valid_index()

    level_df = level_df.iloc[level_start_ind:level_end_ind+1]
    
    #print("Size of level_df: ", len(level_df))
    level_time_secs = (level_df.iloc[-1]['Timestamp'] - level_df.iloc[0]['Timestamp'])/1000
    level_gsr_mean = level_df['GSR Conductance CAL'].mean()
    level_hr_mean = level_df['Heart rate'].mean()

    gsr_df = pd.DataFrame(level_df['GSR Conductance CAL'])
    gsr_df = gsr_df[gsr_df['GSR Conductance CAL'].notna()]
    gsr_list = gsr_df['GSR Conductance CAL'].tolist()
    if(len(gsr_list) > 0):
        signals, info = nk.eda_process(gsr_list, sampling_rate=128)
        level_gsr_peaks_rate = len(info['SCR_Peaks']) / level_time_secs
        #level_gsr_mean_peak_amplitude = sum(info['SCR_Amplitude'])/len(info['SCR_Amplitude'])
        #print(type(info['SCR_Amplitude']))
        level_gsr_mean_peak_amplitude = np.nansum(info['SCR_Amplitude'])/ np.sum(~np.isnan(info['SCR_Amplitude'])) 
        if(debug): 
            # Extract clean EDA and SCR features
            cleaned = signals["EDA_Clean"]
            #features = [info["SCR_Onsets"], info["SCR_Peaks"], info["SCR_Recovery"]]
            features = [info["SCR_Peaks"]]
            #plot = nk.events_plot(features, cleaned, color=['red'])           
            #print(info)
            #print('level_gsr_mean_peak_amplitude', level_gsr_mean_peak_amplitude)

    else:
        #print("No GSR data in this level for this participant")
        level_gsr_peaks_rate = None
        level_gsr_mean_peak_amplitude = None
    #level_gsr_

    rr_intervals_df = pd.DataFrame(level_df['R-R interval'])
    rr_intervals_df = rr_intervals_df[rr_intervals_df['R-R interval'].notna()]
    #convert column to list
    rr_intervals_list = rr_intervals_df['R-R interval'].tolist()
    #print("Size of rr intervals list: ", len(rr_intervals_list))
    if(len(rr_intervals_list) > 0):
        #rr_intervals_df.describe()
        #rr_intervals_df[0]
        time_domain_features = get_time_domain_features(rr_intervals_list)
        freq_domain_features =  get_frequency_domain_features(rr_intervals_list, method='lomb')
        #df[df[column].notna()][column]
        level_rmssd = time_domain_features['rmssd']
        level_sdnn = time_domain_features['sdnn']
        level_mean_nni = time_domain_features['mean_nni']
        level_hf_n = freq_domain_features['hfnu']
        level_lf_n = freq_domain_features['lfnu']
        level_lf_hf_ratio = freq_domain_features['lf_hf_ratio']
    else:
        #print("No RR intervals data in this level for this participant")
        level_rmssd = None
        level_sdnn = None
        level_mean_nni = None
        level_hf_n = None
        level_lf_n = None
        level_lf_hf_ratio = None

    #Calculate total contact time 
    level_contact_df = level_df[level_df['MarkerName'] == 'BuzzWireHit']
    total_contact_time = 0
    for i in range(1,len(level_contact_df)): 
        #print('Currently processing ', mistakeTimeStampsdf.iloc[i])
        timeDiff = level_contact_df.iloc[i]['Timestamp'] - level_contact_df.iloc[i-1]['Timestamp']
        #print('Difference with last time stamp- ', timeDiff)
        if(timeDiff < minTimeDiff): #mistake detected if within the timeDiff threshold
            total_contact_time+=timeDiff
        else:
            total_contact_time+=25
    level_total_contact_time_secs = total_contact_time/1000
    return level_time_secs, level_total_contact_time_secs, level_gsr_mean, level_hr_mean, level_gsr_peaks_rate, level_gsr_mean_peak_amplitude, level_rmssd, level_sdnn, level_mean_nni, level_hf_n, level_lf_n, level_lf_hf_ratio



In [108]:
#Event Analysis
#==============

def processEvents(start_signal, end_signal, sensordata_df, level, participant_id, baseline_hr, baseline_gsr, baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude, baseline_rmssd, baseline_sdnn, baseline_mean_nni, baseline_lf_n, baseline_hf_n, baseline_lf_hf_ratio, debug=False):
    eventsDF = pd.DataFrame(columns=['Participant_ID', 'Level', 'Mistake_Number', 'Mistake_Time_Stamp', 'Baseline_HR', 'Baseline_GSR', 'Baseline_GSR_Mean_Peak_Amplitude', 'Baseline_RMSSD', 'Baseline_SDNN', 'Baseline_Mean_NNI', 'Baseline_LFN', 'Baseline_HFN', 'Baseline_LFHF_Ratio', 'Event_GSR', 'Event_HR', 'Event_RMSSD', 'Event_SDNN', 'Event_Mean_NNI', 'Event_LFN', 'Event_HFN', 'Event_LFHF_Ratio', 'Event_GSR_Peaks_Count', 'Event_GSR_Peak_Amplitude'])
    if(debug):
        print("Processing events in Level: ", level)
    if end_signal != None:
        level_signal_start_time = sensordata_df[sensordata_df['MarkerName'] == start_signal]['Timestamp'].values[0]
        next_level_signal_start_time =  sensordata_df[sensordata_df['MarkerName'] == end_signal]['Timestamp'].values[0]
        sensordata_df = sensordata_df[sensordata_df['Timestamp'].between(level_signal_start_time, next_level_signal_start_time)]
    else:
        #level_signal_start_time = sensordata_df[sensordata_df['MarkerName'] == start_signal]['Timestamp'].values[0]
        sensordata_df = sensordata_df.iloc[sensordata_df.where(sensordata_df['MarkerName'] == start_signal).first_valid_index():]
       

    sensordata_df.reset_index(inplace = True) #As the indices from the main df is carried over
    
    first_right_press_ind = sensordata_df.where(sensordata_df['MarkerName'] == 'RightSwitchPressed').first_valid_index()
    sensordata_df.iloc[first_right_press_ind:].replace(to_replace ="LeftSwitchPressed", value ="", inplace=True) #Remove left presses after right presses

    level_start_ind = sensordata_df.where(sensordata_df['MarkerName'] == 'LeftSwitchPressed').last_valid_index()
    level_end_ind = sensordata_df.where(sensordata_df.iloc[level_start_ind:]['MarkerName'] == 'RightSwitchPressed').first_valid_index()

    level_df = sensordata_df.iloc[level_start_ind:level_end_ind+1]

    #print('sensordata_df column names: ', sensordata_df.columns)
    
    #Event related GSR and HR data
    gsr_event_period = 10000 #10 seconds
    hr_event_period = 10000 #10 seconds 
    level_contact_df = level_df[level_df['MarkerName'] == 'BuzzWireHit']
    for i in range(1,len(level_contact_df)): #For each mistake event calculate event related measures for GSR and HR
        try:
            timeDiff = level_contact_df.iloc[i]['Timestamp'] - level_contact_df.iloc[i-1]['Timestamp']
            #print('Difference with last time stamp- ', timeDiff)
            if(timeDiff > minTimeDiff): #mistake detected if within the timeDiff threshold
                mistake_number = i + 1
                contact_start_time = level_contact_df.iloc[i]['Timestamp']
                gsr_end_time = contact_start_time + gsr_event_period
                #if gsr_end_time > level_df['Timestamp'].max():

                hr_end_time = contact_start_time + hr_event_period
                #Find rows between current timestamp and current timestamp + 10000 
                post_event_df = sensordata_df[sensordata_df['Timestamp'].between(contact_start_time, gsr_end_time)]
                #print('post_event_df column names: ', post_event_df.columns)
                event_gsr_mean = post_event_df['GSR Conductance CAL'].mean()
                event_hr_mean = post_event_df['Heart rate'].mean()

                #Post event HR
                event_rr_intervals_df = pd.DataFrame(post_event_df['R-R interval'])
                event_rr_intervals_df = event_rr_intervals_df[event_rr_intervals_df['R-R interval'].notna()]
                #convert column to list
                event_rr_intervals_list = event_rr_intervals_df['R-R interval'].tolist()
                #print("Size of rr intervals list: ", len(rr_intervals_list))
                if(len(event_rr_intervals_list) > 0):
                    time_domain_features = get_time_domain_features(event_rr_intervals_list)
                    freq_domain_features =  get_frequency_domain_features(event_rr_intervals_list, method='lomb')

                    event_rmssd = time_domain_features['rmssd']
                    event_sdnn = time_domain_features['sdnn']
                    event_mean_nni = time_domain_features['mean_nni']
                    event_hf_n = freq_domain_features['hfnu']
                    event_lf_n = freq_domain_features['lfnu']
                    event_lf_hf_ratio = freq_domain_features['lf_hf_ratio']
                else:
                    print("No RR intervals data in this level for this participant")
                    event_rmssd = None
                    event_sdnn = None
                    event_mean_nni = None
                    event_hf_n = None
                    event_lf_n = None
                    event_lf_hf_ratio = None                            

                #Post event GSR
                event_gsr_df = pd.DataFrame(post_event_df['GSR Conductance CAL'])
                event_gsr_df = event_gsr_df[event_gsr_df['GSR Conductance CAL'].notna()]
                #convert column to list
                event_gsr_list = event_gsr_df['GSR Conductance CAL'].tolist()
                if(debug): 
                    print("Size of event gsr list: ", len(event_gsr_list))
                    #print(event_gsr_list)

                if(len(event_gsr_list) > 0):
                    signals, info = nk.eda_process(event_gsr_list, sampling_rate=128)
                    event_gsr_peaks_count = len(info['SCR_Peaks']) 
                    if(info['SCR_Amplitude'].size > 0):
                        event_gsr_peak_amplitude = info['SCR_Amplitude'][0] #Amplitude of first peak
                    else:
                        event_gsr_peak_amplitude = None
                    #event_gsr_mean_peak_amplitude = np.nansum(info['SCR_Amplitude'])/ np.sum(~np.isnan(info['SCR_Amplitude'])) 
                    if(debug): 
                        # Extract clean EDA and SCR features
                        cleaned = signals["EDA_Clean"]
                        #features = [info["SCR_Onsets"], info["SCR_Peaks"], info["SCR_Recovery"]]
                        features = [info["SCR_Peaks"]]
                        #plot = nk.events_plot(features, cleaned, color=['red'])           
                        #print(info)
                        #print('event_gsr_peak_amplitude', event_gsr_peak_amplitude)

                else:
                    print("No GSR data in this event for this participant")
                    event_gsr_peaks_count = None
                    event_gsr_peak_amplitude = None         

                #if(debug):
                    #print("Event: ", mistake_number, " GSR: ", event_gsr_mean, ' Event_GSR_Peaks_Count: ', event_gsr_peaks_count, ' Event_GSR_Peak_Amplitude : ',  event_gsr_peak_amplitude, " HR: ", event_hr_mean, " RMSSD: ", event_rmssd, " SDNN: ", event_sdnn, " Mean NNI: ", event_mean_nni, " HFN: ", event_hf_n, " LFN: ", event_lf_n, " LF/HF Ratio: ", event_lf_hf_ratio)

                #Append data to eventsDF dataframe
                eventsDF = eventsDF.append({'Participant_ID' : participant_id, 'Level' : level, 'Mistake_Number' : mistake_number, 'Mistake_Time_Stamp' : contact_start_time, 'Baseline_HR' : baseline_hr, 'Baseline_GSR' : baseline_gsr, 'Baseline_GSR_Peaks_Rate' : baseline_gsr_peaks_rate, 'Baseline_GSR_Mean_Peak_Amplitude' : baseline_gsr_mean_peak_amplitude, 'Baseline_RMSSD' : baseline_rmssd, 'Baseline_SDNN' : baseline_sdnn, 'Baseline_Mean_NNI' : baseline_mean_nni, 'Baseline_LFN' : baseline_lf_n, 'Baseline_HFN' : baseline_hf_n, 'Baseline_LFHF_Ratio' : baseline_lf_hf_ratio, 'Event_GSR' : event_gsr_mean, 'Event_HR' : event_hr_mean, 'Event_RMSSD' : event_rmssd, 'Event_SDNN' : event_sdnn, 'Event_Mean_NNI' : event_mean_nni, 'Event_LFN' : event_lf_n, 'Event_HFN' : event_hf_n, 'Event_LFHF_Ratio' : event_lf_hf_ratio, 'Event_GSR_Peaks_Count' : event_gsr_peaks_count, 'Event_GSR_Peak_Amplitude' : event_gsr_peak_amplitude}, ignore_index=True)
                
                #if(debug):
                #    print(eventsDF.tail(1))

        except ValueError:
            print("ValueError: ", ValueError)
            print("ValueError at Level: ", level, " Participant: ", participant_id, " Mistake Number: ", mistake_number)                      
            
            # handle ValueError exception
            pass

    if(debug):
        print('Events processed in this level: ', eventsDF.shape[0])
    return eventsDF

In [109]:
def analyzeData(condition, location, processed_file_name, debug=False):
    finalResultsDF = pd.DataFrame(columns=['Participant_ID','Baseline_HR', 'Baseline_GSR', 'Baseline_GSR_Peaks_Rate', 'Baseline_GSR_Mean_Peak_Amplitude', 'Baseline_RMSSD', 'Baseline_SDNN', 'Baseline_Mean_NNI', 'Baseline_LFN', 'Baseline_HFN', 'Baseline_LFHF_Ratio', 'Pretest_Time', 'Pretest_Mistake_Time', 'Pretest_GSR', 'Pretest_HR', 'Pretest_RMSSD', 'Pretest_SDNN', 'Pretest_Mean_NNI', 'Pretest_LFN', 'Pretest_HFN', 'Pretest_LFHF_Ratio', 'Pretest_GSR_Peaks_Rate', 'Pretest_GSR_Mean_Peak_Amplitude', 'Level_1_Time', 'Level_1_Mistake_Time', 'Level_1_HR','Level_1_GSR', 'Level_1_RMSSD', 'Level_1_SDNN', 'Level_1_Mean_NNI', 'Level_1_LFN', 'Level_1_HFN', 'Level_1_LFHF_Ratio', 'Level_1_GSR_Peaks_Rate', 'Level_1_GSR_Mean_Peak_Amplitude', 'Level_2_Time', 'Level_2_Mistake_Time', 'Level_2_HR','Level_2_GSR', 'Level_2_RMSSD', 'Level_2_SDNN', 'Level_2_Mean_NNI', 'Level_2_LFN', 'Level_2_HFN', 'Level_2_LFHF_Ratio', 'Level_2_GSR_Peaks_Rate', 'Level_2_GSR_Mean_Peak_Amplitude','Level_3_Time', 'Level_3_Mistake_Time', 'Level_3_HR','Level_3_GSR', 'Level_3_RMSSD', 'Level_3_SDNN', 'Level_3_Mean_NNI', 'Level_3_LFN', 'Level_3_HFN', 'Level_3_LFHF_Ratio', 'Level_3_GSR_Peaks_Rate', 'Level_3_GSR_Mean_Peak_Amplitude', 'Level_4_Time', 'Level_4_Mistake_Time', 'Level_4_HR','Level_4_GSR', 'Level_4_RMSSD', 'Level_4_SDNN', 'Level_4_Mean_NNI', 'Level_4_LFN', 'Level_4_HFN', 'Level_4_LFHF_Ratio', 'Level_4_GSR_Peaks_Rate', 'Level_4_GSR_Mean_Peak_Amplitude', 'Posttest_Time', 'Posttest_Mistake_Time', 'Posttest_GSR', 'Posttest_HR', 'Posttest_RMSSD', 'Posttest_SDNN', 'Posttest_Mean_NNI', 'Posttest_LFN', 'Posttest_HFN', 'Posttest_LFHF_Ratio', 'Posttest_GSR_Peaks_Rate', 'Posttest_GSR_Mean_Peak_Amplitude'])
    finalEventsDF = pd.DataFrame(columns=['Participant_ID', 'Level', 'Mistake_Number', 'Mistake_Time_Stamp', 'Baseline_HR', 'Baseline_GSR', 'Baseline_GSR_Mean_Peak_Amplitude', 'Baseline_RMSSD', 'Baseline_SDNN', 'Baseline_Mean_NNI', 'Baseline_LFN', 'Baseline_HFN', 'Baseline_LFHF_Ratio', 'Event_GSR', 'Event_HR', 'Event_RMSSD', 'Event_SDNN', 'Event_Mean_NNI', 'Event_LFN', 'Event_HFN', 'Event_LFHF_Ratio', 'Event_GSR_Peaks_Count', 'Event_GSR_Peak_Amplitude'])
    
    file_list = glob.glob(condition + location)
    for file in file_list:
        file_name = file[file.rindex('\\')+1:]
        participant_id = file_name[file_name.find('_') + 1:file_name.find('.')][:6] #Remove all characters other than the participant id
        print('Currently processing sensor data from participant id - ', participant_id)

        #sensorDataDF_test = pd.read_csv('VR_BuzzwireStudy/BuzzwireStudy/Sensor Data/Final/016_2D0402.csv')#pd.read_csv(file)
        sensorDataDF_test = pd.read_csv(file)
        sensorDataDF_test = sensorDataDF_test.iloc[27:] #Ignore iMotions metadata in the first 27 rows
        headers = sensorDataDF_test.iloc[0] #Take the resulting df's first row 
        sensorDataDF_test  = pd.DataFrame(sensorDataDF_test.values[1:], columns=headers)
        sensorDataDF_test = sensorDataDF_test[['Timestamp', 'Heart rate','R-R interval','MarkerName','GSR Conductance CAL']].copy()
        sensorDataDF_test['Timestamp'] = sensorDataDF_test['Timestamp'].astype(float)
        sensorDataDF_test['Heart rate'] = sensorDataDF_test['Heart rate'].astype(float)
        sensorDataDF_test['R-R interval'] = sensorDataDF_test['R-R interval'].astype(float)
        sensorDataDF_test['GSR Conductance CAL'] = sensorDataDF_test['GSR Conductance CAL'].astype(float)
        #for i in range(1,len(sensorDataDF_test)): 
        #    currMarkerName = sensorDataDF_test.iloc[i]['MarkerName']
        #    if pd.isnull(currMarkerName): continue

        #Baseline
        baseline_start_time = sensorDataDF_test[sensorDataDF_test['MarkerName'] == 'baseline_started']['Timestamp'].values[0]
        baseline_end_time =  sensorDataDF_test[sensorDataDF_test['MarkerName'] == 'baseline_over']['Timestamp'].values[0]
        baseline_gsr_df = pd.DataFrame(sensorDataDF_test[sensorDataDF_test['Timestamp'].between(baseline_start_time, baseline_end_time)]['GSR Conductance CAL'])
        #print(baseline_gsr_df.columns)
        baseline_gsr_df = baseline_gsr_df[baseline_gsr_df['GSR Conductance CAL'].notna()]
        baseline_gsr = baseline_gsr_df['GSR Conductance CAL'].mean()
        baseline_gsr_list = baseline_gsr_df['GSR Conductance CAL'].tolist()
        if(len(baseline_gsr_list) > 0):
            signals, info = nk.eda_process(baseline_gsr_list, sampling_rate=128)
            baseline_gsr_peaks_rate = len(info['SCR_Peaks']) / 180 #3 min baseline = 180 secs
            baseline_gsr_mean_peak_amplitude = np.nansum(info['SCR_Amplitude'])/ np.sum(~np.isnan(info['SCR_Amplitude'])) 
        else:
            print("No GSR data in this level for this participant")
            baseline_gsr_peaks_rate = None
            baseline_gsr_mean_peak_amplitude = None
            
        #print(baseline_gsr_df.mean())
        #baseline_gsr_list = baseline_gsr_df['GSR Conductance CAL'].tolist()
        #signals, info = nk.eda_process(baseline_gsr_list, sampling_rate=128)
        #print("Baseline GSR - ", baseline_gsr)
        baseline_hr_df = sensorDataDF_test[sensorDataDF_test['Timestamp'].between(baseline_start_time, baseline_end_time)]['Heart rate']
        baseline_hr = baseline_hr_df.mean()
        baseline_rr_intervals_df = pd.DataFrame(sensorDataDF_test[sensorDataDF_test['Timestamp'].between(baseline_start_time, baseline_end_time)]['R-R interval'])
        baseline_rr_intervals_df = baseline_rr_intervals_df[baseline_rr_intervals_df['R-R interval'].notna()]
        #convert column to list
        baseline_rr_intervals_list = baseline_rr_intervals_df['R-R interval'].tolist()
    
        baseline_time_domain_features = get_time_domain_features(baseline_rr_intervals_list)
        baseline_freq_domain_features =  get_frequency_domain_features(baseline_rr_intervals_list, sampling_frequency=1)
        baseline_rmssd = baseline_time_domain_features['rmssd']
        baseline_sdnn = baseline_time_domain_features['sdnn']
        baseline_mean_nni = baseline_time_domain_features['mean_nni']
        baseline_hf_n = baseline_freq_domain_features['hfnu']
        baseline_lf_n = baseline_freq_domain_features['lfnu']
        baseline_lf_hf_ratio = baseline_freq_domain_features['lf_hf_ratio']

        #Analyze per participant
        pretest_time_secs, pretest_total_contact_time_secs, pretest_gsr_mean, pretest_hr_mean, pretest_gsr_peaks_rate, pretest_gsr_mean_peak_amplitude, pretest_rmssd, pretest_sdnn, pretest_mean_nni, pretest_hf_n, pretest_lf_n, pretest_lf_hf_ratio  = processLevel('pre_test_started', 'level_1_started', sensorDataDF_test, level='Pretest', debug=debug)
        level_1_time_secs, level_1_total_contact_time_secs, level_1_gsr_mean , level_1_hr_mean, level_1_gsr_peaks_rate, level_1_gsr_mean_peak_amplitude, level_1_rmssd, level_1_sdnn, level_1_mean_nni, level_1_hf_n, level_1_lf_n, level_1_lf_hf_ratio = processLevel('level_1_started', 'level_2_started', sensorDataDF_test, level='Level 1',debug=debug)
        level_2_time_secs, level_2_total_contact_time_secs, level_2_gsr_mean , level_2_hr_mean, level_2_gsr_peaks_rate, level_2_gsr_mean_peak_amplitude, level_2_rmssd, level_2_sdnn, level_2_mean_nni, level_2_hf_n, level_2_lf_n, level_2_lf_hf_ratio = processLevel('level_2_started', 'level_3_started', sensorDataDF_test, level='Level 2',debug=debug)
        level_3_time_secs, level_3_total_contact_time_secs, level_3_gsr_mean , level_3_hr_mean, level_3_gsr_peaks_rate, level_3_gsr_mean_peak_amplitude, level_3_rmssd, level_3_sdnn, level_3_mean_nni, level_3_hf_n, level_3_lf_n, level_3_lf_hf_ratio = processLevel('level_3_started', 'level_4_started', sensorDataDF_test, level='Level 3',debug=debug)
        level_4_time_secs, level_4_total_contact_time_secs, level_4_gsr_mean , level_4_hr_mean, level_4_gsr_peaks_rate, level_4_gsr_mean_peak_amplitude, level_4_rmssd, level_4_sdnn, level_4_mean_nni, level_4_hf_n, level_4_lf_n, level_4_lf_hf_ratio = processLevel('level_4_started', 'post_test_started', sensorDataDF_test, level='Level 4',debug=debug)
        posttest_time_secs, posttest_total_contact_time_secs, posttest_gsr_mean, posttest_hr_mean, posttest_gsr_peaks_rate, posttest_gsr_mean_peak_amplitude, posttest_rmssd, posttest_sdnn, posttest_mean_nni, posttest_hf_n, posttest_lf_n, posttest_lf_hf_ratio= processLevel('post_test_started', None, sensorDataDF_test, level='Posttest',debug=debug)

        #Analyze each event per participant
        finalEventsDF = finalEventsDF.append(processEvents('pre_test_started', 'level_1_started', sensorDataDF_test, level='Pretest', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)
        finalEventsDF = finalEventsDF.append(processEvents('level_1_started', 'level_2_started', sensorDataDF_test, level='Level 1', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)
        finalEventsDF = finalEventsDF.append(processEvents('level_2_started', 'level_3_started', sensorDataDF_test, level='Level 2', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)
        finalEventsDF = finalEventsDF.append(processEvents('level_3_started', 'level_4_started', sensorDataDF_test, level='Level 3', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)
        finalEventsDF = finalEventsDF.append(processEvents('level_4_started', 'post_test_started', sensorDataDF_test, level='Level 4', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)
        finalEventsDF = finalEventsDF.append(processEvents('post_test_started', None, sensorDataDF_test, level='Posttest', participant_id=participant_id, debug=debug, baseline_hr = baseline_hr, baseline_gsr = baseline_gsr, baseline_gsr_peaks_rate = baseline_gsr_peaks_rate, baseline_gsr_mean_peak_amplitude = baseline_gsr_mean_peak_amplitude, baseline_rmssd = baseline_rmssd, baseline_sdnn = baseline_sdnn, baseline_mean_nni = baseline_mean_nni, baseline_lf_n = baseline_lf_n, baseline_hf_n = baseline_hf_n, baseline_lf_hf_ratio = baseline_lf_hf_ratio), ignore_index=True)

        #Add to final results df
        #print('',{'Participant_ID' : participant_id,'Baseline_HR' : baseline_hr, 'Baseline_GSR' : baseline_gsr, 'Pretest_Time' : pretest_time_secs, 'Pretest_Mistake_Time' : pretest_total_contact_time_secs, 'Pretest_GSR' : pretest_gsr, 'Pretest_HR' : pretest_hr, 'Level_1_Time' : level_1_time_secs, 'Level_1_Mistake_Time' : level_1_total_contact_time_secs, 'Level_1_HR' : level_1_hr,'Level_1_GSR' : level_1_gsr, 'Level_2_Time' : level_2_time_secs, 'Level_2_Mistake_Time' : level_2_total_contact_time_secs, 'Level_2_HR' : level_2_hr,'Level_2_GSR' : level_2_gsr, 'Level_3_Time' : level_3_time_secs, 'Level_3_Mistake_Time' : level_3_total_contact_time_secs, 'Level_3_HR' : level_3_hr,'Level_3_GSR' : level_3_gsr, 'Level_4_Time' : level_4_time_secs, 'Level_4_Mistake_Time' : level_4_total_contact_time_secs, 'Level_4_HR' : level_4_hr,'Level_4_GSR' : level_4_gsr,'Posttest_Time' : posttest_time_secs, 'Posttest_Mistake_Time' : posttest_total_contact_time_secs, 'Posttest_GSR' : posttest_gsr, 'Posttest_HR' : posttest_hr})
        finalResultsDF = finalResultsDF.append(
            {'Participant_ID' : participant_id,'Baseline_HR' : baseline_hr, 'Baseline_GSR' : baseline_gsr, 'Baseline_GSR_Peaks_Rate' : baseline_gsr_peaks_rate, 'Baseline_GSR_Mean_Peak_Amplitude' : baseline_gsr_mean_peak_amplitude, 'Baseline_RMSSD' : baseline_rmssd, 'Baseline_SDNN' : baseline_sdnn, 'Baseline_Mean_NNI' : baseline_mean_nni, 'Baseline_LFN' : baseline_lf_n, 'Baseline_HFN' : baseline_hf_n, 'Baseline_LFHF_Ratio' : baseline_lf_hf_ratio, 
            'Pretest_Time' : pretest_time_secs, 'Pretest_Mistake_Time' : pretest_total_contact_time_secs, 'Pretest_GSR' : pretest_gsr_mean, 'Pretest_HR' : pretest_hr_mean, 'Pretest_RMSSD' : pretest_rmssd, 'Pretest_SDNN' : pretest_sdnn, 'Pretest_Mean_NNI' : pretest_mean_nni, 'Pretest_LFN' : pretest_lf_n, 'Pretest_HFN' : pretest_hf_n, 'Pretest_LFHF_Ratio' : pretest_lf_hf_ratio, 'Pretest_GSR_Peaks_Rate' : pretest_gsr_peaks_rate, 'Pretest_GSR_Mean_Peak_Amplitude' : pretest_gsr_mean_peak_amplitude, 
            'Level_1_Time' : level_1_time_secs, 'Level_1_Mistake_Time' : level_1_total_contact_time_secs, 'Level_1_HR' : level_1_hr_mean,'Level_1_GSR' : level_1_gsr_mean, 'Level_1_RMSSD' : level_1_rmssd, 'Level_1_SDNN' : level_1_sdnn, 'Level_1_Mean_NNI' : level_1_mean_nni, 'Level_1_LFN' : level_1_lf_n, 'Level_1_HFN' : level_1_hf_n, 'Level_1_LFHF_Ratio' : level_1_lf_hf_ratio, 'Level_1_GSR_Peaks_Rate' : level_1_gsr_peaks_rate, 'Level_1_GSR_Mean_Peak_Amplitude' : level_1_gsr_mean_peak_amplitude, 
            'Level_2_Time' : level_2_time_secs, 'Level_2_Mistake_Time' : level_2_total_contact_time_secs, 'Level_2_HR' : level_2_hr_mean,'Level_2_GSR' : level_2_gsr_mean, 'Level_2_RMSSD' : level_2_rmssd, 'Level_2_SDNN' : level_2_sdnn, 'Level_2_Mean_NNI' : level_2_mean_nni, 'Level_2_LFN' : level_2_lf_n, 'Level_2_HFN' : level_2_hf_n, 'Level_2_LFHF_Ratio' : level_2_lf_hf_ratio, 'Level_2_GSR_Peaks_Rate' : level_2_gsr_peaks_rate, 'Level_2_GSR_Mean_Peak_Amplitude' : level_2_gsr_mean_peak_amplitude, 
            'Level_3_Time' : level_3_time_secs, 'Level_3_Mistake_Time' : level_3_total_contact_time_secs, 'Level_3_HR' : level_3_hr_mean,'Level_3_GSR' : level_3_gsr_mean, 'Level_3_RMSSD' : level_3_rmssd, 'Level_3_SDNN' : level_3_sdnn, 'Level_3_Mean_NNI' : level_3_mean_nni, 'Level_3_LFN' : level_3_lf_n, 'Level_3_HFN' : level_3_hf_n, 'Level_3_LFHF_Ratio' : level_3_lf_hf_ratio, 'Level_3_GSR_Peaks_Rate' : level_3_gsr_peaks_rate, 'Level_3_GSR_Mean_Peak_Amplitude' : level_3_gsr_mean_peak_amplitude, 
            'Level_4_Time' : level_4_time_secs, 'Level_4_Mistake_Time' : level_4_total_contact_time_secs, 'Level_4_HR' : level_4_hr_mean,'Level_4_GSR' : level_4_gsr_mean, 'Level_4_RMSSD' : level_4_rmssd, 'Level_4_SDNN' : level_4_sdnn, 'Level_4_Mean_NNI' : level_4_mean_nni, 'Level_4_LFN' : level_4_lf_n, 'Level_4_HFN' : level_4_hf_n, 'Level_4_LFHF_Ratio' : level_4_lf_hf_ratio, 'Level_4_GSR_Peaks_Rate' : level_4_gsr_peaks_rate, 'Level_4_GSR_Mean_Peak_Amplitude' : level_4_gsr_mean_peak_amplitude, 
            'Posttest_Time' : posttest_time_secs, 'Posttest_Mistake_Time' : posttest_total_contact_time_secs, 'Posttest_GSR' : posttest_gsr_mean, 'Posttest_HR' : posttest_hr_mean, 'Posttest_RMSSD' : posttest_rmssd, 'Posttest_SDNN' : posttest_sdnn, 'Posttest_Mean_NNI' : posttest_mean_nni, 'Posttest_LFN' : posttest_lf_n, 'Posttest_HFN' : posttest_hf_n, 'Posttest_LFHF_Ratio' : posttest_lf_hf_ratio, 'Posttest_GSR_Peaks_Rate' : posttest_gsr_peaks_rate, 'Posttest_GSR_Mean_Peak_Amplitude' : posttest_gsr_mean_peak_amplitude},
             ignore_index=True)
             
        
    finalEventsDF.to_csv((processed_file_name + '_events_' + condition + '.csv'), sep=',')
    finalResultsDF.to_csv((processed_file_name + condition + '.csv'), sep=',')
    print("Analysis complete")


In [110]:
analyzeData('VR',"_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\Final\\*.csv", "finalResults_")
analyzeData('Ph', "_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\Final\\*.csv", "finalResults_")
analyzeData('VR',"_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\FinalRemoved\\*.csv", "finalResults_part_invalid_")
analyzeData('Ph', "_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\FinalRemoved\\*.csv", "finalResults_part_invalid_")

#analyzeData('vr', "_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\Final\\002_2D0804.csv", "debug", debug=True) #For debugging specific participant data

Currently processing sensor data from participant id -  2D0804


C:\Users\au657021\AppData\Local\Temp\ipykernel_18556\2914428093.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  analyzeData('VR',"_BuzzwireStudy\\BuzzwireStudy\\Sensor Data\\Final\\*.csv", "finalResults_")
C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\hrvanalysis\extract_features.py:123: RuntimeWarning: divide by zero encountered in true_divide
  heart_rate_list = np.divide(60000, nn_intervals)
C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\numpy\core\_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 256 is greater than input length  = 183, using nperseg = 183
  warnings.warn('nperse

ValueError:  <class 'ValueError'>
ValueError at Level:  Posttest  Participant:  2D0804  Mistake Number:  2
ValueError:  <class 'ValueError'>
ValueError at Level:  Posttest  Participant:  2D0804  Mistake Number:  4
ValueError:  <class 'ValueError'>
ValueError at Level:  Posttest  Participant:  2D0804  Mistake Number:  7
ValueError:  <class 'ValueError'>
ValueError at Level:  Posttest  Participant:  2D0804  Mistake Number:  9
Currently processing sensor data from participant id -  2D0803


C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 256 is greater than input length  = 185, using nperseg = 185
  warnings.warn('nperseg = {0:d} is greater than input length '


Currently processing sensor data from participant id -  2D0802


...

No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data i

C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 256 is greater than input length  = 15, using nperseg = 15
  warnings.warn('nperseg = {0:d} is greater than input length '


No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data in this event for this participant
No GSR data i

C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\astropy\timeseries\periodograms\lombscargle\core.py:232: RuntimeWarning: divide by zero encountered in double_scalars
  df = 1.0 / baseline / samples_per_peak
C:\Users\au657021\AppData\Local\Continuum\anaconda3\envs\buzz_env\lib\site-packages\astropy\timeseries\periodograms\lombscargle\core.py:246: RuntimeWarning: invalid value encountered in multiply
  return minimum_frequency + df * np.arange(Nf)


No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this participant
No RR intervals data in this level for this part